In [12]:
import pandas as pd
import numpy as np
from rich import print
from icecream import ic
import os
from pathlib import Path

from embed_sources import bert_sentence_embed, create_sentence_embedding
from sentence_transformers import SentenceTransformer

In [ ]:
model = SentenceTransformer("all-mpnet-base-v2")

In [9]:
filepath = Path.cwd().joinpath("embeddings", "so_database_app.csv")
so_df = pd.read_csv(filepath)
so_df.head()

,input_text,output_text,category
0,"python's inspect.getfile returns ""<string>""<p>...",<p><code>&lt;string&gt;</code> means that the ...,python
1,Passing parameter to function while multithrea...,<p>Try this and note the difference:</p>\n<pre...,python
2,How do we test a specific method written in a ...,"<p>Duplicate of <a href=""https://stackoverflow...",python
3,how can i remove the black bg color of an imag...,<p>The alpha channel &quot;disappears&quot; be...,python
4,How to extract each sheet within an Excel file...,<p>You need to specify the <code>index</code> ...,python


In [10]:
so_df.category.unique()

array(['python', 'html', 'r', 'css'], dtype=object)